a

stud_math.csv
Описание датасета
Посмотрим на переменные, которые содержит датасет:

1 school — аббревиатура школы, в которой учится ученик

2 sex — пол ученика ('F' - женский, 'M' - мужской)

3 age — возраст ученика (от 15 до 22)

4 address — тип адреса ученика ('U' - городской, 'R' - за городом)

5 famsize — размер семьи('LE3' <= 3, 'GT3' >3)

6 Pstatus — статус совместного жилья родителей ('T' - живут вместе 'A' - раздельно)

7 Medu — образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

8 Fedu — образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)

9 Mjob — работа матери ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)

10 Fjob — работа отца ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)

11 reason — причина выбора школы ('home' - близость к дому, 'reputation' - репутация школы, 'course' - образовательная программа, 'other' - другое)

12 guardian — опекун ('mother' - мать, 'father' - отец, 'other' - другое)

13 traveltime — время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.)

14 studytime — время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов)

15 failures — количество внеучебных неудач (n, если 1<=n<=3, иначе 0)

16 schoolsup — дополнительная образовательная поддержка (yes или no)

17 famsup — семейная образовательная поддержка (yes или no)

18 paid — дополнительные платные занятия по математике (yes или no)

19 activities — дополнительные внеучебные занятия (yes или no)

20 nursery — посещал детский сад (yes или no)

21 higher — хочет получить высшее образование (yes или no)

22 internet — наличие интернета дома (yes или no)

23 romantic — в романтических отношениях (yes или no)

24 famrel — семейные отношения (от 1 - очень плохо до 5 - очень хорошо)

25 freetime — свободное время после школы (от 1 - очень мало до 5 - очень мого)

26 goout — проведение времени с друзьями (от 1 - очень мало до 5 - очень много)

27 health — текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо)

28 absences — количество пропущенных занятий

29 score — баллы по госэкзамену по математике

In [ ]:
from scipy.stats import ttest_ind
from itertools import combinations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

students = pd.read_csv('stud_math.csv')

%matplotlib inline

In [ ]:
students.info()

In [ ]:
students.describe(include=[object])

In [ ]:
# pre functions


'''отображает количество уникальных значений в столбце, частоту встречаний значений, 
info() про столбец, стат.данные для номинативных значений (count, unique, top, freq),
график частоты встречающихся значений, 
если есть порпущенные значения - количество таких записей,
таблицу всех записей с пропущенными значениями'''


def column_info(column):
    print('Столбец ', column)
    print('---------------------------')
    print('Уникальных значений', students[column].nunique())

    display(students[column].value_counts())
    students.loc[:, [column]].info()

    if students[column].dtype == object:
        display(students[column].describe(include=[object]))

    sns.countplot(x=column, data=students)
    plt.show()
    st_with_na = students.loc[students[column].isna()]
    if len(st_with_na) > 0:
        print('Пропущено значений ', len(st_with_na))
        display(st_with_na)


'''отображает стат.данные для числовых столбцов'''


def numeric_col_info(column):
    st = students[column].value_counts().reset_index()

    display(students[column].describe())


'''для числовых столбцов: отображает границы выбросов, к-тво выбросов'''


def has_outliers(column, df=students):
    perc25 = df[column].quantile(0.25)
    perc75 = df[column].quantile(0.75)
    IQR = perc75 - perc25

    print('---------------------------')
    print('Границы выбросов: ', perc25-1.5*IQR, ', ', perc75+1.5*IQR)
    print('---------------------------')

    ok_values_cnt = (df[column].loc[df[column].between(
        perc25-1.5*IQR, perc75+1.5*IQR)]).count()
    outliers_cnt = df[column].count() - ok_values_cnt

    print('---------------------------')
    print('Количество выбросов', outliers_cnt)
    print('---------------------------')


'''заменяет NaN значение в столбце на моду (самое частое значение, которое встречается в столбце)'''


def replace_nan_by_mode(column):
    students[column].fillna(students[column].mode()[0], inplace=True)
    column_info(column)


'''заменяет NaN значение в столбце на медиану '''


def replace_nan_by_median(column):
    students[column].fillna(students[column].median(), inplace=True)
    column_info(column)


'''для столбца, который содержит номинативные числовые данные переводит записи в строки
для удобтва использования функции describe(include=[object])'''


def nominative_column_to_str(column):
    students[column] = students[column].apply(
        lambda x: x if pd.isna(x) else str(int(x)))


'''строит боксплот для зависимости столбца и успешности (score), 
учитывается 10 самых часто встречающихся категорий значений для столбца
строит график частоты встречания самих значений столбца (на том же срезе выборки)'''


def get_boxplot(column):

    fig = plt.figure()

    fig, axes = plt.subplots(figsize=(6, 4))
    sns.boxplot(x=column,
                y='score',
                data=students.loc[students.loc[:, column].isin(
                    students.loc[:, column].value_counts().index[:10])],
                ax=axes)
    axes.set_title('Boxplot for '+column)
    plt.xticks(rotation=45)
    plt.show()

    sns.countplot(x=column, data=students.loc[students.loc[:, column].isin(
        students.loc[:, column].value_counts().index[:10])])
    plt.show()


'''т-тест (ttest_ind) для определения значимости столбца по отношению к успешности'''


def get_stat_dif(column):
    cols = students.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(students.loc[students.loc[:, column] == comb[0], 'score'],
                     students.loc[students.loc[:, column] == comb[1], 'score']).pvalue \
                <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

In [ ]:
'''заменить NaN значения, сохраняя пропорции распределения значений 
в полностью заполненных строках данных (если всего значений 2)'''
'''a/b = x/y; x+y=n1, n1 - количество NaN записей; a, b - частота встречания записей top_val, sec_val'''
'''без глобальных переменных не нашла как выкрутиться'''

import math
top_val = 0
sec_val = 0
cnt = 0


def replace_nan_in_propotion(column):
    global top_val
    global sec_val
    global cnt

    top_val = st[column].value_counts().reset_index()['index'][0]
    sec_val = st[column].value_counts().reset_index()['index'][1]
    top_fr = st[column].value_counts().reset_index()[column][0]
    sec_fr = st[column].value_counts().reset_index()[column][1]

    n1 = len(st.loc[st[column].isna()])
    x = math.ceil(top_fr*n1 / (top_fr + sec_fr))
    cnt = x

    '''x - запись в ячейке
    cnt - сколько NaN-записей надо заменить на топ-значение (top_val)
    остальные NaN-записи заменить на второе значение (sec_val)'''
    def change_x_by_proportion(x):
        global top_val
        global sec_val
        global cnt
        if ((pd.isna(x)) and (cnt > 0)):
            #print('cnt ', cnt, top_val)
            cnt -= 1
            return top_val
        if ((pd.isna(x)) and (cnt <= 0)):
            #print('sec val ', cnt, sec_val)
            return sec_val
        return x

    st[column] = st[column].apply(change_x_by_proportion)

числовые столбцы : age, absences, score

другие столбцы с числовыми значениями содержат номинатвные данные - числа, которые означают тот или иной признак. часто для таких значений встречаться в описании стат.данные, которые выводятся для числовых признаков, игнорируем, при необходимости значения будут переписываться в object и тогда стат.данные будут иметь осмысленный вид
(речь про функцию describe(include=[object])

In [ ]:
# анализируем на выбросы числовые столбцы: age, score, absences
column_info('age')
numeric_col_info('age')
has_outliers('age')

есть выброс, на значении 22 года, удаляем его

In [ ]:
students = students.loc[students.age <= 21]

In [ ]:
column_info('score')
numeric_col_info('score')
has_outliers('score')

удаляем строки, в которых значение score не указано

In [ ]:
students = students.loc[~students.score.isna()]

In [ ]:
column_info('absences')
numeric_col_info('absences')
has_outliers('absences')

есть выбросы, обсудим их чуть ниже

Посмотрим на связь состояния здоровья и количества пропусков занятий. Возможно, студенты с плохим состоянием здоровья пропускают занятия чаще?

In [ ]:
fig, axes = plt.subplots(figsize=(6, 4))
sns.boxplot(x='health',
            y='absences',
            data=students.loc[students.absences > 20],
            ax=axes)
axes.set_title('Boxplot for health')
plt.xticks(rotation=45)
plt.show()

нет, состояние здоровья на прямую не связано. Видно, что те, у кого, состояние здоровья 1,2 пропустили зантий меньше, чем те, у кого состояние здоровья оценено на 4,5

In [ ]:
fig, axes = plt.subplots(figsize=(14, 4))
sns.boxplot(x='absences',
            y='score',
            data=students.loc[students.absences > 20],
            ax=axes)
axes.set_title('Boxplot for absences')
plt.xticks(rotation=45)
plt.show()

Удалять выбросы тут не хочется, количество записей 17 (почти 4.4% выборки), к тому же в значениях выбросов 3 (из почти 100) соответствуют самым высоким баллам за экзамен (больше Q3; >70)

пропуски заполним медианой

In [ ]:
replace_nan_by_median('absences')

In [ ]:
column_info('school')
column_info('sex')
column_info('age')

In [ ]:
# school, sex, age - не имеют пропусков в значениях

In [ ]:
column_info('address')

In [ ]:
#address - traveltime
address_traveltime = students.groupby(by=['address', 'traveltime'])[
    'address'].agg('count')
display(address_traveltime)

address_traveltime.plot(kind='bar')

что живующие за городом, что в городе имеют признак traveltime всех значений (1,2,3,4 встречаются в каждой выборке).
Посмотрим, какие значения traveltime у тех, записей, где пропущено значение address

In [ ]:
students.loc[students.address.isna()].traveltime.hist(range=(1, 4))

встречаются значения 1, 2, есть пропущенные, к тому же U является модой для всей выборки в целом, заменяем пропущенные значения в столбце address на U

In [ ]:
students.address = students.address.apply(lambda x: 'U' if pd.isna(x) else x)

In [ ]:
column_info('famsize')

для пропущенных значений famsize введем значение UNK unknown

In [ ]:
students.famsize = students.famsize.apply(lambda x: 'UNK' if pd.isna(x) else x)

In [ ]:
column_info('Pstatus')

аналогично, для пропущенных значений Pstatus введем значение UNK unknown

In [ ]:
students.Pstatus = students.Pstatus.apply(lambda x: 'UNK' if pd.isna(x) else x)

In [ ]:
#nominative_cols = ['Medu', 'Fedu', 'traveltime', 'studytime', 'failures',
#                   'famrel', 'freetime', 'goout', 'health']

#for col in nominative_cols:
#    nominative_column_to_str(col)

In [ ]:
column_info('Medu')

In [ ]:
column_info('Mjob')

Значение Medu (Fedu) будем заполнять с учетом значений Mjob (Fjob):
Для каждой сферы работы найдем наиболее часто встречаемую категорию образования.

В свою очередь, пропуски в Mjob (Fjob) заполним значеним other - это является и модой для выборки мам и пап, и по здравому смыслу хорошо подходит.

In [ ]:
job_edu_m = students.groupby(by=['Mjob', 'Medu'])['Medu'].agg('count')
display(job_edu_m)

#  чаще всего встречается такое соотношение между работа-образование среди мам:
# at_home  - 1
# health   - 4
# other    - 2
# services - 3
# teacher  - 4
# на основании этого делаем замену для недостающих данных в колонке Medu


def correctMedu_byMjob():
    students.loc[(students.Medu.isna()) & (
        students.Mjob == 'at_home'), 'Medu'] = 1
    students.loc[(students.Medu.isna()) & (
        students.Mjob == 'health'), 'Medu'] = 4
    students.loc[(students.Medu.isna()) & (
        students.Mjob == 'other'), 'Medu'] = 2
    students.loc[(students.Medu.isna()) & (
        students.Mjob == 'services'), 'Medu'] = 3
    students.loc[(students.Medu.isna()) & (
        students.Mjob == 'teacher'), 'Medu'] = 4


correctMedu_byMjob()

In [ ]:
column_info('Mjob')

In [ ]:
column_info('Fedu')

предполагаю, что значение 40 является опечаткой, заменяю на значение 4

In [ ]:
students.loc[students.Fedu == 40, 'Fedu'] = 4

In [ ]:
column_info('Fedu')

In [ ]:
job_edu_f = students.groupby(by=['Fjob', 'Fedu'])['Fedu'].agg('count')
display(job_edu_m)

#  чаще всего встречается такое соотношение между работа-образование среди пап:
# at_home  - 1
# health   - 4
# other    - 2
# services - 3
# teacher  - 4
# на основании этого делаем замену для недостающих данных в колонке Fedu


def correctFedu_byFjob():
    students.loc[(students.Fedu.isna()) & (
        students.Fjob == 'at_home'), 'Fedu'] = 1
    students.loc[(students.Fedu.isna()) & (
        students.Fjob == 'health'), 'Fedu'] = 4
    students.loc[(students.Fedu.isna()) & (
        students.Fjob == 'other'), 'Fedu'] = 2
    students.loc[(students.Fedu.isna()) & (
        students.Fjob == 'services'), 'Fedu'] = 3
    students.loc[(students.Fedu.isna()) & (
        students.Fjob == 'teacher'), 'Fedu'] = 4


correctFedu_byFjob()

In [ ]:
# дополним недостающее записи в Mjob(Fjob)
students.Mjob = students.Mjob.apply(lambda x: 'other' if pd.isna(x) else x)
students.Fjob = students.Fjob.apply(lambda x: 'other' if pd.isna(x) else x)
correctFedu_byFjob()
correctMedu_byMjob()

In [ ]:
column_info('reason')

мода - courses, по здравому смыслу пропущенные значения хочу заполнить значениями other

In [ ]:
students.reason = students.reason.apply(lambda x: 'other' if pd.isna(x) else x)

In [ ]:
column_info('guardian')

In [ ]:
sns.countplot(
    x='Pstatus', data=students.loc[students.guardian.isna()], hue='famsize')

Наиболее часто для строк с пропущенным guardian признак проживания -  T, и семья больше чем 3 человека,  что скорее всего значит, что семья полная и проживает вместе.
заменим пропуски на mother, как на моду


In [ ]:
replace_nan_by_mode('guardian')

In [ ]:
column_info('traveltime')

как уже смотрели раннее - значение 1 является модой для двух групп: и для тех, кто живет в городе, и для тех, кто за городом. Заменяем пропуски на моду выборки

In [ ]:
replace_nan_by_mode('traveltime')

In [ ]:
column_info('studytime')

заменим пропуски на значение моды

In [ ]:
replace_nan_by_mode('studytime')

In [ ]:
column_info('failures')

In [ ]:
get_boxplot('failures')

признак failures довольно противоречивый, значение 0 может соответствовать как отсуствию неудач, так и их количеству 4+, что видно на боксплоте. 

рассмотрим для записей с пропущенным значением failures как распредела успещность студентов

In [ ]:
sns.countplot(x='score', data=students.loc[students.failures.isna()])

на графике видно, что нет явных смещений в успешности студентов. Чтоб не искажать данные введем новую категорию для NaN в столбце failures: -1

In [ ]:
students.failures.fillna(-1, inplace=True)

In [ ]:
get_boxplot('failures')

In [ ]:
column_info('schoolsup')
column_info('famsup')

In [ ]:
sns.boxplot(x='famsup', y='score', data=students)

по заполненным данным не очевидно, что есть связь с успешностью

In [ ]:
sns.boxplot(x='famsup', y='score', data=students, hue='famrel')

исходя из этого графика видно, что однозначной прямой связи хороших отношений в семье, помощи с уроками и успешности нет. студенты с отношениями плохими (1, оранж цвет) имеют помощь с уроками от семьи, и имеют успешность выше чем студенты с хорошими отнощениями (5, кор.цв.) и с помощью с уроками и без нее. С отношениями 2 (зел. цв.) успешность носит обратный характер зависимости с помощью с уроками в семье.
Вероятно этот признак не будет включен в нашу модель.
Заменим пропуски на моду.

In [ ]:
replace_nan_by_mode('famsup')

In [ ]:
sns.boxplot(x='schoolsup', y='score', data=students, hue='school')

школа MS не организовывает помощь студентам, а для школы GP видим, что средний бал у тех, кто берет доп занятия хоть и ниже, но в группе, где студенты не берут доп занятия размах баллов (нижний ус - который соответствует неуспешности) большой. Вероятно эти данные не будут иметь влияния в нашей модели. Заменим пропуски на моду.

In [ ]:
replace_nan_by_mode('schoolsup')

In [ ]:
column_info('paid')
sns.boxplot(x='paid', y='score', data=students)

In [ ]:
column_info('activities')
sns.boxplot(x='activities', y='score', data=students)

In [ ]:
column_info('nursery')
sns.boxplot(x='nursery', y='score', data=students)

Похоже, что параметры paid, activities, nursery не войдут в модель. Заполним пропуски значениями моды.
Возможно хорошим вариантом будет заполнить пропуски пропорционально текущим имеющимся распределениям параметра.

In [ ]:
replace_nan_by_mode('paid')
replace_nan_by_mode('activities')
replace_nan_by_mode('nursery')

In [ ]:
column_info('studytime, granular')

возможно studytime, granular это категориальный признак. введем для пропущенных значений новую категорию 0

In [ ]:
students['studytime, granular'].fillna(0, inplace=True)

In [ ]:
column_info('higher')

In [ ]:
sns.countplot(x='score', data=students.loc[~students.higher.isna()])

In [ ]:
sns.countplot(x='score', data=students.loc[students.higher.isna()])

однозначных выводов сделать не получается, введем новую категорию -1

In [ ]:
students.higher.fillna(-1, inplace=True)

In [ ]:
column_info('internet')

заменим пропуски в  столбце internet на моду (можно заменить так же и пропорционально replace_nan_in_proportion())

In [ ]:
replace_nan_by_mode('internet')

In [ ]:
column_info('romantic')

по здравому смыслу и "по моде" хочется заменить пропуски на значение no

In [ ]:
replace_nan_by_mode('romantic')

In [ ]:
column_info('famrel')

вероятно значение -1 это опечатка, заменим на 1

In [ ]:
students.loc[students.famrel == -1, 'famrel'] = 1

пропущенных значений 27, заменить на моду - плохой выбор, введем новую категорию -1, можно было б думать в сторону замены пропусков в пропорциональных частях, но кажется этот параметр может не войти в модель.

In [ ]:
students.famrel.fillna(-1, inplace=True)

In [ ]:
column_info('freetime')

In [ ]:
column_info('goout')

In [ ]:
column_info('health')

Для признаков freetime, goout, healt попробуем заполнить пропуски модами, оценим т-стастики, и сравним, что будет если заполним их новыми введенными категориями (-1)

In [ ]:
# заполнение модой
replace_nan_by_mode('freetime')
replace_nan_by_mode('goout')
replace_nan_by_mode('health')

In [ ]:
# заполнение категорией -1
#students.freetime.fillna(-1, inplace=True)
#students.goout.fillna(-1, inplace=True)
#students.health.fillna(-1, inplace=True)


In [ ]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
            'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
            'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
            'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
            'freetime', 'goout', 'health']:
    get_boxplot(col)

есть следуюшие предположения (если про столбец ничего не написано, то данные "интересные", с какими-то подозрениями на зависимости):

school: в одной школе размах данных большой, но и выборка студентов существенно больше чем во второй школе. возможно параметр не войдет в модель

famsize: семьи больше 3 человек - существенная часть выборки, имеет больший размах данных по успеваемости, чем другие категории, что логично, возможно этот параметр так же не будет входить в модель

Pstatus: имеет существенный перевес в выборке значение Т, при том, что на боксплоте видно, что срдняя успеваемость не варьируется в группах (A, UNK, T), размах по величине схож в группах (UNK, T). Возможно параметр не войдет в модель

reason: слабо варьируется средняя успеваемость в выделенных группах, возможно параметр не войдет в модель

guardian: тоже слабо варьируется средняя успеваемость, размах в группе с признаком mother больше, но и сама выборка существенно больше других. Возможно параметр не пойдет в модель

traveltime: слабо варьируется средняя успеваемость, размах с признаком 1 больше, как и сама выборка студентов с призаком 1. Возможно параметр не войдет в модель

famsup: Возможно не войдет в модель, на боксплоте не очевидны зависимости
paid: Возможно не войдет в модель, на боксплоте не очевидны зависимости
activities: Возможно не войдет в модель, на боксплоте не очевидны зависимости
nursery: Возможно не войдет в модель, на боксплоте не очевидны зависимости

intenet: количественно очень смещенная выборка по признаку (yes/no). разные размеры рахмахов. Возможно параметр не войдет в модель

famrel: слабо варьируется средняя успеваемость. Возможно параметр не войдет в модель

постановка теста "чем заполнить значения freetime, goout, healt": перезагрузить ядро юпитера, выполнить все стрOки до строкИ замены, дальше выполнить желанные замены пропусков, и сравнить полученные в двух разных "заполнениях" результируещие списки значимых критериев (функция get_stat_dif).

заполненно модами:

Найдены статистически значимые различия для колонки sex

Найдены статистически значимые различия для колонки address

Найдены статистически значимые различия для колонки Medu

Найдены статистически значимые различия для колонки Mjob

Найдены статистически значимые различия для колонки failures

Найдены статистически значимые различия для колонки higher

Найдены статистически значимые различия для колонки romantic



заполнено новыми категориями -1:

Найдены статистически значимые различия для колонки sex

Найдены статистически значимые различия для колонки address

Найдены статистически значимые различия для колонки Medu

Найдены статистически значимые различия для колонки Mjob

Найдены статистически значимые различия для колонки failures

Найдены статистически значимые различия для колонки higher

Найдены статистически значимые различия для колонки romantic


списки совпадают

дабы не плодить сущности для замены выбираем значения мод

In [ ]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
            'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
            'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
            'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
            'freetime', 'goout', 'health']:
    get_stat_dif(col)

In [ ]:
correlation1 = students[['score', 'age', 'absences']].corr()
sns.heatmap(correlation1, annot=True, cmap='YlGn')

age, absences имеют корреляцию с успеваемостью -0.15 и ~0.1(округлили). Добавим их в модель.

ради исследовательствого интереса предположим, что sudytime, granular это числовое знаечние

In [ ]:
correlation2 = students[['score', 'age',
                         'studytime, granular', 'absences']].corr()
sns.heatmap(correlation2, annot=True, cmap='YlGn')

из интересного: видно слабую корреляцию между studytime, granular(sg) и absences. Возможно неописанный признак sg имеет физический смысл частичной учебы то дома то в школе в связи с пропусками занятий, но помним, что мы предположили что значения -3,-6,-9,-12,0 сейчас несут числовой смысл, не категориальный. это всего лишь наши допущения.

In [ ]:
students.info()

Промежуточные выводы написаны по ходу блокнота и в выполняемых командах

Резюме

Основываясь на т-тесте в модель к рассмотрению предлагается включить следующие столбцы:
sex
address
Medu
Mjob
failures
higher
romantic

так же включить в модель age и absences. Последний признак довольно противоречивый, считаю важным оставить его без отсечения выбросов, принимать дальнейшее решение по его видоизменению (убрать выбросы) или полному удалению, с учетом дальнейшего хода исследований.



In [ ]:
sns.distplot(students.score)

график получился бимодальным из-за наличия большого количества записей с успеваемостью 0. Значение 0 не является выбросом. Визуально "колокол" распределения успеваемости похож на нормальное распределение. Распрделение возраста (jointplot ниже)тоже имеет форму смещенного колокола. 

Целесообразность считать что это нормальное распределение сомнительна из соображений здравого смысла: поступающие в школу на первый год обучения имеют(в среднем) константный возраст, в процессе обучения учащиеся могут отчисляться, и соответственно на следующих курсах (более старших студентов) их может учиться меньше. НО, тут надо больше знать о том как собирались данные этой выборки, да и вообще скорее надо учитывать, что сама величина возраст - Дискретная, а не непрерывная, но модель это модель...

In [ ]:
sns.jointplot(x='age',
              y='score',
              data=students)

In [ ]:
sns.jointplot(x='absences',
              y='score',
              data=students)

посмотрим "что будет если"
обрежем выбросы по параметру absences и посмотрим на распределние успеваемости/пропусков, так красивее видно распределение параметра absences, который словами можно описать, что среди студентов существует тенденция не пропускать занятия.

In [ ]:
stud_absenses_otliers = students.loc[students.absences <= 20]
sns.jointplot(x='absences',
              y='score',
              data=stud_absenses_otliers)

Далее быстрый эксперимент на коленке "а что будет если" из существуюшей выборки мы б удаляли все строки, где встречались какие-то незаполненные значения

In [ ]:
stud_super_cutted = pd.read_csv('stud_math.csv')
columns = stud_super_cutted.columns
columns
for col in columns:
    stud_super_cutted = stud_super_cutted.loc[~stud_super_cutted[col].isna()]

stud_super_cutted.info()

осталось 97 записей

In [ ]:
corr_s_c1 = stud_super_cutted[['score', 'age', 'absences']].corr()
sns.heatmap(corr_s_c1, annot=True, cmap='YlGn')

тут корреляции глобально - как были незначительными, так и остались, даже еще меньше видна связь успеваемости и пропусков (ну маленькая выборка, как ни крути)

In [ ]:
def get_stat_dif_s_c(column):
    cols = stud_super_cutted.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(stud_super_cutted.loc[stud_super_cutted.loc[:, column] == comb[0], 'score'],
                     stud_super_cutted.loc[stud_super_cutted.loc[:, column] == comb[1], 'score']).pvalue \
                <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

In [ ]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
            'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
            'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
            'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
            'freetime', 'goout', 'health']:
    get_stat_dif_s_c(col)

конечно и тут другой набор ответов выходит. видно, что он сильно отличается, если изменить параметр alpha 0.1, то еще к списку добавить колонка failures.
В общем это очень не похоже на то, какие выводы были сделаны по большей выборке, где я максимально вдумчиво старалась дополнить пробелы в данных. 

ВЫВОД: надо стараться дополнять данные, а не безжалостно их выбрасывать

тест

заменили некоторые категориальные данные (2 значения) с сохранением пропорции распределения значений.

famsup, schoolsup, paid, activities, nursery, interner

провели т-тест. получили такой же набор значимых параметров.

значит такая предобработка данных тоже имеет право на существование

In [ ]:
students = pd.read_csv('stud_math.csv')


'''NAN fill'''
students = students.loc[students.age <= 21]
students = students.loc[~students.score.isna()]
replace_nan_by_median('absences')
students.address = students.address.apply(
    lambda x: 'U' if pd.isna(x) else x)
students.famsize = students.famsize.apply(
    lambda x: 'UNK' if pd.isna(x) else x)
students.Pstatus = students.Pstatus.apply(
    lambda x: 'UNK' if pd.isna(x) else x)
correctMedu_byMjob()
students.loc[students.Fedu == 40, 'Fedu'] = 4
correctFedu_byFjob()
students.Mjob = students.Mjob.apply(lambda x: 'other' if pd.isna(x) else x)
students.Fjob = students.Fjob.apply(lambda x: 'other' if pd.isna(x) else x)
correctFedu_byFjob()
correctMedu_byMjob()
students.reason = students.reason.apply(
    lambda x: 'other' if pd.isna(x) else x)
replace_nan_by_mode('guardian')
replace_nan_by_mode('traveltime')
replace_nan_by_mode('studytime')
students.failures.fillna(-1, inplace=True)

replace_nan_in_propotion('famsup')
replace_nan_in_propotion('schoolsup')
replace_nan_in_propotion('paid')
replace_nan_in_propotion('activities')
replace_nan_in_propotion('nursery')

students['studytime, granular'].fillna(0, inplace=True)
students.higher.fillna(-1, inplace=True)

replace_nan_in_propotion('internet')


replace_nan_by_mode('romantic')
students.loc[students.famrel == -1, 'famrel'] = 1
students.famrel.fillna(-1, inplace=True)
replace_nan_by_mode('freetime')
replace_nan_by_mode('goout')
replace_nan_by_mode('health')


for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
            'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
            'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
            'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
            'freetime', 'goout', 'health']:
    get_stat_dif(col)

In [ ]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
            'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
            'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
            'studytime, granular', 'higher', 'internet', 'romantic', 'famrel',
            'freetime', 'goout', 'health']:
    get_stat_dif(col)